In [1]:
# Start final project import modules
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
from sklearn.cross_validation import cross_val_score
%matplotlib inline
import sys
import string
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder 
from sklearn import feature_extraction
from __future__ import print_function
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier


In [2]:
# read in the datasets
 
#jd = pd.read_csv('job_data_dict_30_sample.csv')
#jd = pd.read_csv('job_data_dict_sal_weighted_30_sample.csv')
#jd = pd.read_csv('job_data_dict_all_sal_weighted.csv')
jd = pd.read_csv('job_data_dict_all_sal_weighted_major.csv')

In [3]:
#1. Explore the dataset: 
    # There is a mix of string and numeric data. There are some variables that can be converted to dummy or binary values.
    # While there are no NAs there are unknown values in some of the string data which may need to be treated as NAs.

#jd.describe(include='all')
jd.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 9716 entries, 0 to 9715
Data columns (total 12 columns):
Unnamed: 0          9716 non-null int64
major_des           9716 non-null object
jobs                9716 non-null object
jobtitle            9716 non-null object
onetcode            9716 non-null object
major_code          9716 non-null int64
dictionary          9716 non-null object
p_an                9716 non-null float64
p_an_normal         9716 non-null float64
lowct               9716 non-null int64
major_dictionary    9716 non-null object
uid                 9716 non-null int64
dtypes: float64(2), int64(4), object(6)
memory usage: 986.8+ KB


In [4]:
jd.head()

,Unnamed: 0,major_des,jobs,jobtitle,onetcode,major_code,dictionary,p_an,p_an_normal,lowct,major_dictionary,uid
0,2,Management,computer and information systems managers,director of it planning development deployment,11-3021.00,11,director planning development deployment direc...,110000.0,0.097679,0,computer information systems managers computer...,1
1,6,Management,financial managers branch or department,accounting manager supervisor,11-3031.02,11,accounting manager supervisor accounting manag...,45000.0,0.031779,0,financial managers branch department financial...,2
2,16,Management,human resources managers,human resources manager,11-3040.00,11,human resources manager human resources manage...,91197.5,0.078616,0,human resources managers human resources manag...,3
3,28,Management,treasurers and controllers,corporate controller,11-3031.01,11,corporate controller corporate controller year...,115000.0,0.102748,0,treasurers controllers treasurers controllers ...,4
4,31,Management,financial managers branch or department,accounting manager supervisor,11-3031.02,11,accounting manager supervisor accounting manag...,82500.0,0.069798,0,financial managers branch department financial...,5


In [5]:

cat_columns = ['major_des', 'jobs','jobtitle','dictionary','major_dictionary', 'p_an_normal', 'lowct']
major_feat_col = ['major_dictionary', 'p_an_normal']   
feat_col = ['dictionary', 'p_an_normal']
string_variables = ['jobs']

jd_2 = jd[cat_columns]
jd_2.head()

,major_des,jobs,jobtitle,dictionary,major_dictionary,p_an_normal,lowct
0,Management,computer and information systems managers,director of it planning development deployment,director planning development deployment direc...,computer information systems managers computer...,0.097679,0
1,Management,financial managers branch or department,accounting manager supervisor,accounting manager supervisor accounting manag...,financial managers branch department financial...,0.031779,0
2,Management,human resources managers,human resources manager,human resources manager human resources manage...,human resources managers human resources manag...,0.078616,0
3,Management,treasurers and controllers,corporate controller,corporate controller corporate controller year...,treasurers controllers treasurers controllers ...,0.102748,0
4,Management,financial managers branch or department,accounting manager supervisor,accounting manager supervisor accounting manag...,financial managers branch department financial...,0.069798,0


In [6]:
# subset the data to the major groups
jd_3 = jd_2[jd_2.major_des=='Business and Financial Operations']
jd_3 = jd_3[jd_3.lowct==0]

In [7]:
jd_3.jobs.value_counts().min()

28

In [8]:
#df_numerical = jd_2.copy()
#encoder_dict = {}

#for col in string_variables:
#    le = preprocessing.LabelEncoder()
#    le.fit(jd_2[col])
#    encoder_dict[col] = le
#    df_numerical[col] = le.transform(jd_2[col])

#df_numerical.head()

In [9]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(jd_3[feat_col], jd_3['jobs'], random_state=1)

In [10]:
#feat = list(df_numerical.dictionary.values.flatten())
#type(feat)
#feat = unicode(feat, 'utf-8')

In [11]:
#df_numerical.dictionary[0]
#import sys  

#reload(sys)  
#sys.setdefaultencoding('utf8')
#y_test


In [12]:
X_test.shape

(230L, 2L)

In [13]:
X_train.shape

(687L, 2L)

In [14]:
type(X_train)

numpy.ndarray

In [15]:
from sklearn.feature_extraction.text import CountVectorizer
vect = CountVectorizer(decode_error = 'ignore')
#vect = CountVectorizer()
#vect.fit(X_train)
#vect.transform(X_train)

In [16]:
#vect.get_feature_names()
# learn vocabulary and create document-term matrix in a single step
train_dtm = vect.fit_transform(X_train[:,0])
train_dtm

<687x1323 sparse matrix of type '<type 'numpy.int64'>'
	with 6483 stored elements in Compressed Sparse Row format>

In [17]:
# transform testing data into a document-term matrix
test_dtm = vect.transform(X_test[:,0])
type(test_dtm)

scipy.sparse.csr.csr_matrix

In [18]:
train_features = vect.get_feature_names()
len(train_features)

1323

In [19]:
train_features = train_features + list('s')

In [20]:
len(train_features)

1324

In [21]:
train_sal = np.array(X_train[:,1])
train_sal = train_sal[...,None]
type(train_sal)

numpy.ndarray

In [22]:
test_sal = np.array(X_test[:,1])
test_sal = test_sal[...,None]
type(test_sal)

numpy.ndarray

In [23]:
train_dtm_2 = np.array(train_dtm.todense())
type(train_dtm_2)

numpy.ndarray

In [24]:
test_dtm_2 = np.array(test_dtm.todense())

In [25]:
test_dtm_2.shape

(230L, 1323L)

In [26]:
train_dtm_2.shape

(687L, 1323L)

In [27]:
train_dtm2 = np.append(train_dtm_2, train_sal,1)

In [28]:
type(train_dtm2)
train_dtm2.shape

(687L, 1324L)

In [29]:
test_dtm2 = np.append(test_dtm_2, test_sal, 1)

In [30]:
# convert train_dtm to a regular array
#train_arr = train_dtm2.toarray()
#train_arr

In [31]:
# exercise: create a DataFrame of tokens with their counts.
train_token_counts = pd.DataFrame({'token':train_features, 'count':np.sum(train_dtm2, axis=0)})
train_token_counts.sort_index(by='count', ascending=False)

,count,token
1110,499,senior
68,451,analyst
9,397,accountant
497,296,financial
1160,253,staff
738,197,manager
12,188,accounting
293,187,controller
1008,179,recruiter
113,156,auditor


In [86]:
# train a Naive Bayes model using train_dtm
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(train_dtm2, y_train)

C:\Anaconda\lib\site-packages\sklearn\naive_bayes.py:427: DeprecationWarning: Implicitly casting between incompatible kinds. In a future numpy release, this will raise an error. Use casting="unsafe" if this is intentional.
  self.feature_count_ += safe_sparse_dot(Y.T, X)


MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [87]:
# make predictions on test data using test_dtm
preds = nb.predict(test_dtm2)
preds

array(['management analysts', 'business operations specialists',
       'accountants', 'accountants', 'management analysts',
       'business operations specialists',
       'purchasing agents except wholesale retail and farm products',
       'accountants',
       'purchasing agents except wholesale retail and farm products',
       'auditors', 'accountants', 'personnel recruiters',
       'financial analysts', 'accountants', 'accountants', 'accountants',
       'personnel recruiters', 'personnel recruiters',
       'personnel recruiters', 'financial analysts', 'accountants',
       'logisticians', 'accountants', 'financial analysts',
       'financial analysts', 'financial analysts', 'personnel recruiters',
       'accountants', 'logisticians', 'auditors', 'management analysts',
       'accountants', 'accountants', 'accountants', 'personnel recruiters',
       'business operations specialists', 'loan officers',
       'financial analysts',
       'purchasing agents except wholesale r

In [88]:
from sklearn import metrics
print(metrics.accuracy_score(y_test, preds))
print(metrics.confusion_matrix(y_test, preds))

0.860869565217
[[83  0  0  5  0  0  2  0  0]
 [ 0 21  0  0  0  0  0  0  0]
 [ 2  0  6  1  0  1  3  0  0]
 [ 2  0  0 20  0  0  1  0  0]
 [ 1  0  0  1  6  0  0  0  0]
 [ 0  0  0  0  0  6  0  0  1]
 [ 2  0  1  6  0  0 23  0  0]
 [ 1  0  0  0  0  0  0 21  0]
 [ 0  0  1  0  0  0  0  1 12]]


In [89]:
# predict (poorly calibrated) probabilities and calculate AUC
#probs = nb.predict_proba(test_dtm2)[:, 1]
#probs
#print(metrics.roc_auc_score(y_test, probs))

In [90]:
# exercise: show the message text for the false positives
#X_test[y_test < preds]

In [91]:

# exercise: show the message text for the false negatives
#X_test[y_test > preds]

In [92]:
from sklearn.metrics import classification_report 

print(classification_report(y_test, preds) )

             precision    recall  f1-score   support

accountants       0.91      0.92      0.92        90
   auditors       1.00      1.00      1.00        21
business operations specialists       0.75      0.46      0.57        13
financial analysts       0.61      0.87      0.71        23
loan officers       1.00      0.75      0.86         8
logisticians       0.86      0.86      0.86         7
management analysts       0.79      0.72      0.75        32
personnel recruiters       0.95      0.95      0.95        22
purchasing agents except wholesale retail and farm products       0.92      0.86      0.89        14

avg / total       0.87      0.86      0.86       230



In [32]:
# use SVM
from sklearn import svm
svc = svm.SVC(kernel='linear', probability = True)

svc.fit(train_dtm2, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0, degree=3, gamma=0.0,
  kernel='linear', max_iter=-1, probability=True, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [33]:
# make predictions on test data using test_dtm
preds = svc.predict(test_dtm2)
preds

array(['financial analysts', 'business operations specialists',
       'accountants', 'accountants', 'management analysts',
       'business operations specialists',
       'purchasing agents except wholesale retail and farm products',
       'accountants',
       'purchasing agents except wholesale retail and farm products',
       'auditors', 'accountants', 'personnel recruiters', 'accountants',
       'accountants', 'accountants', 'accountants', 'personnel recruiters',
       'personnel recruiters', 'personnel recruiters',
       'financial analysts', 'accountants', 'logisticians', 'accountants',
       'financial analysts', 'financial analysts', 'accountants',
       'purchasing agents except wholesale retail and farm products',
       'accountants', 'logisticians', 'auditors', 'management analysts',
       'accountants', 'accountants', 'accountants', 'personnel recruiters',
       'business operations specialists', 'loan officers',
       'financial analysts', 'accountants', 'acco

In [34]:
from sklearn import metrics
print(metrics.accuracy_score(y_test, preds))
print(metrics.confusion_matrix(y_test, preds))

0.865217391304
[[86  0  0  4  0  0  0  0  0]
 [ 1 20  0  0  0  0  0  0  0]
 [ 1  0  7  1  0  1  3  0  0]
 [ 2  0  0 20  1  0  0  0  0]
 [ 1  0  0  1  6  0  0  0  0]
 [ 0  0  0  0  0  7  0  0  0]
 [ 4  0  2  6  0  0 20  0  0]
 [ 1  0  0  0  0  0  0 21  0]
 [ 1  0  1  0  0  0  0  0 12]]


In [95]:
# try using all of the data

In [96]:
train_dtm = vect.fit_transform(jd_3['dictionary'])
train_dtm_2 = np.array(train_dtm.todense())
type(train_dtm_2)

numpy.ndarray

In [97]:
train_sal = np.array(jd_3['p_an_normal'])
train_sal = train_sal[...,None]
type(train_sal)

numpy.ndarray

In [98]:
train_dtm2 = np.append(train_dtm_2, train_sal,1)

In [99]:
# train a Naive Bayes model using train_dtm
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb.fit(train_dtm2, jd_3['jobs'])

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [100]:
preds = nb.predict(train_dtm2)
preds.shape

(917L,)

In [101]:

print(classification_report(jd_3['jobs'], preds) )

             precision    recall  f1-score   support

accountants       0.99      0.94      0.97       387
   auditors       1.00      0.99      0.99        81
business operations specialists       0.98      0.89      0.93        54
financial analysts       0.81      0.96      0.88       109
loan officers       1.00      0.98      0.99        41
logisticians       0.96      0.96      0.96        28
management analysts       0.89      0.95      0.92        82
personnel recruiters       0.98      1.00      0.99        92
purchasing agents except wholesale retail and farm products       0.98      0.98      0.98        43

avg / total       0.96      0.96      0.96       917



In [102]:

target_predicted = nb.predict(train_dtm2)
target_prob = nb.predict_proba(train_dtm2)

#print(target_test)
#print(target_predicted)
#print(target_prob.mean())

pd.crosstab(jd_3['jobs'], target_predicted, rownames=['True'], colnames=['Predicted'], margins=True)

Predicted,accountants,auditors,business operations specialists,financial analysts,loan officers,logisticians,management analysts,personnel recruiters,purchasing agents except wholesale retail and farm products,All
True,,,,,,,,,,
accountants,364,0,1,20,0,0,2,0,0,387
auditors,1,80,0,0,0,0,0,0,0,81
business operations specialists,0,0,48,0,0,1,4,1,0,54
financial analysts,0,0,0,105,0,0,4,0,0,109
loan officers,0,0,0,1,40,0,0,0,0,41
logisticians,0,0,0,0,0,27,0,0,1,28
management analysts,1,0,0,3,0,0,78,0,0,82
personnel recruiters,0,0,0,0,0,0,0,92,0,92
purchasing agents except wholesale retail and farm products,0,0,0,0,0,0,0,1,42,43


In [103]:
from sklearn.cross_validation import cross_val_score
cross_val_score(nb, train_dtm, jd_3['jobs'], cv=5, scoring='accuracy').mean()

0.85927054090864208